In [ ]:
import pandas as pd
import numpy as np

### Importando o dataset

In [ ]:
# Lendo o csv supermarket
smt=pd.read_csv("/content/supermarket.csv")
# Substituindo "t" por 1 e "?" por 0
smt.replace("t", 1, inplace=True) # Substituindo "t" por 1 e "?" por 0
smt.replace("?", 0, inplace=True)
smt

,department1,department2,department3,department4,department5,department6,department7,department8,department9,'grocery misc',...,department208,department209,department210,department211,department212,department213,department214,department215,department216,total
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,high
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,low
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,low
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,low
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,low
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4622,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,low
4623,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,high
4624,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,low
4625,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,low


In [ ]:
# Determinando os valores minimos
SUP_min = 0.6
CONF_min = 0.56
CONVI_min = 1.20
LIFT_min = 1
total=len(smt)
lista_itens = list(smt.columns)

# Regra de associação A=>B


---
## Calculando o Suporte de cada item



In [ ]:
def calcula_suporte(item):
  contagem_item = smt[item].value_counts().get(1,0) # Busca os valores 1("t")
  return contagem_item / total



dicionario_suportes={} # Criando um dicionario para os suportes
for i in smt.columns: # Percorrendo as colunas do csv
    if i != "total": # Descartando a coluna total
      suporte = calcula_suporte(i) # Calculnado o suporte para "i"
      if suporte >= SUP_min: # Garante que o suporte é maior que o suporte minimo
        dicionario_suportes[f"{i}"]="Suporte: ",suporte

#  Exibindo mostrando os suportes
for item, suporte in dicionario_suportes.items():
  print(f"{item} - {suporte} - \n")

'bread and cake' - ('Suporte: ', 0.719688783228874) - 

'baking needs' - ('Suporte: ', 0.6040631078452561) - 

milk-cream - ('Suporte: ', 0.635184784957856) - 

fruit - ('Suporte: ', 0.640155608385563) - 

vegetables - ('Suporte: ', 0.6399394856278366) - 



Primeiramente filtramos os suportes, selecionando apenas aqueles maiores que 0.6


---
### Calculando união A,B

In [ ]:
def calcula_uni(A, B):
    uniao = smt.value_counts([A, B]).get((1, 1), 0) # Selecionando os valores em que A e B venderam juntos
    return uniao / total # Calculando a frequencia


dicionario_uniao = {}
for i, item_i in enumerate(dicionario_suportes.keys()): # Enumerate verifica a posição do item em estudo
        for j, item_j in enumerate(dicionario_suportes.keys()):
          if i > j:  # Garante que não estamos repetindo combinações
                try:
                    uniao = calcula_uni(item_i, item_j)
                    if uniao != 0: # Verifica se o suporte da união é mais que zero
                        dicionario_uniao[len(dicionario_uniao)] = {'Item 1': item_i, 'Item 2': item_j, 'União': uniao} # Adicionando a um dicionario
                except ValueError:
                    pass
df_uniao=pd.DataFrame.from_dict(dicionario_uniao, orient='index') # Transforma as chaves dos dicionarios em indices
df_uniao

,Item 1,Item 2,União
0,'baking needs','bread and cake',0.473525
1,milk-cream,'bread and cake',0.505079
2,milk-cream,'baking needs',0.412146
3,fruit,'bread and cake',0.502485
4,fruit,'baking needs',0.410633
5,fruit,milk-cream,0.440458
6,vegetables,'bread and cake',0.496650
7,vegetables,'baking needs',0.421223
8,vegetables,milk-cream,0.437649
9,vegetables,fruit,0.476983


Aqui estruturamos um DataFrame com a uniao dos suportes A e B

---
### Calculando a confiança A=>B

In [ ]:

def calcula_conf (A, B):
  return calcula_uni(A, B) / calcula_suporte(A) # Calcula o suporte da uniao de A,B e divide pelo suporte de A


dicionario_conf={}
for i, item_i in enumerate(dicionario_suportes.keys()):
          for j, item_j in enumerate(dicionario_suportes.keys()):
              if  i > j:  # Garante que não estamos repetindo combinações
                  try:
                    confianca = calcula_conf(item_i,item_j)
                    if confianca != 0 and confianca>=CONF_min:
                          dicionario_conf[len(dicionario_conf)] = {'Item 1': item_i, 'Item 2': item_j, 'Confiança': confianca}
                  except ZeroDivisionError:
                      pass
df_conf = pd.DataFrame.from_dict(dicionario_conf, orient='index') # Transforma as chaves dos dicionarios em indices

df_conf

,Item 1,Item 2,Confiança
0,'baking needs','bread and cake',0.783900
1,milk-cream,'bread and cake',0.795168
2,milk-cream,'baking needs',0.648860
3,fruit,'bread and cake',0.784943
4,fruit,'baking needs',0.641458
5,fruit,milk-cream,0.688049
6,vegetables,'bread and cake',0.776089
7,vegetables,'baking needs',0.658224
8,vegetables,milk-cream,0.683891
9,vegetables,fruit,0.745356


  A partir da confiança podemos assumir que clientes que compraram itens de panificação (“baking needs”), compraram junto pão e bolo (“bread and cake”) 78% das vezes. Da mesma forma que, quem comprou creme de leite (“milk-cream”) comprou também pão e bolo em 79,5% das ocasiões, porém quando analisamos creme de leite e itens de panificação esse número cai para 64%. Isso mostra que as pessoas optam por levar esses itens juntos com bastante frequência, o que faz sentido pela similaridade dos itens.
  Outra relação forte é entre frutas (“fruits”), pão e bolo, tendo uma confiança de 78%.  Para frutas e creme de leite este valor corresponde a 68%. Ademais, o cliente que comprou vegetais (“vegetables”) comprou também pão e bolo 77% das vezes, similar para vegetais e frutas, ocorrendo em 74% das ocasiões.
	Portanto, pela conexão dos itens e alta porcentagem de vendas juntos seria conveniente ao cliente que estes itens estivessem próximos uns aos outros, facilitando a compra.


---
### Calculando o Lift A=>B

In [ ]:
def calcula_lift(A, B):
   return calcula_uni(A,B) / (calcula_suporte(A)*calcula_suporte(B))



dicionario_lift = {}
for i, item_i in enumerate(dicionario_suportes.keys()):
        for j, item_j in enumerate(dicionario_suportes.keys()):
            if i > j:  # Garante que não estamos repetindo combinações
              try:
                lift_value = calcula_lift(item_i, item_j)
                if lift_value>=LIFT_min:
                    dicionario_lift[len(dicionario_lift)] = {'Item 1': item_i, 'Item 2': item_j, 'Lift': lift_value}
              except ZeroDivisionError:
                lift_value = 0
df_lift = pd.DataFrame.from_dict(dicionario_lift, orient='index')


df_lift



,Item 1,Item 2,Lift
0,'baking needs','bread and cake',1.089221
1,milk-cream,'bread and cake',1.104878
2,milk-cream,'baking needs',1.074160
3,fruit,'bread and cake',1.090670
4,fruit,'baking needs',1.061906
5,fruit,milk-cream,1.083226
6,vegetables,'bread and cake',1.078368
7,vegetables,'baking needs',1.089660
8,vegetables,milk-cream,1.076680
9,vegetables,fruit,1.164336


Um lift próximo de 1 indica que não há uma associação forte entre os itens. Portanto, no caso do lift de 1.10 para a regra de associação entre creme de leite, pão e bolo, isso sugere que o fato de um cliente comprar creme de leite não aumenta expressivamente a chance de comprar pão e bolo. Da mesma forma, um lift de 1.16 entre vegetais e frutas também sugere que a associação entre esses itens não é muito forte

---
## Convicção A=>B

In [ ]:
def calcula_convi(A,B):
  return (1-calcula_suporte(B)) / (1-calcula_conf(A,B))
dicionario_convi = {}
for i, item_i in enumerate(dicionario_suportes.keys()):
        for j, item_j in enumerate(dicionario_suportes.keys()):
            if i > j:  # Garante que não estamos repetindo combinações
              try:
                convi_value = calcula_convi(item_i, item_j)
                if convi_value >= CONVI_min:
                    dicionario_convi[len(dicionario_convi)] = {'Item 1': item_i, 'Item 2': item_j, 'Convicção': convi_value}
              except ZeroDivisionError:
                convi_value = 0
df_convi = pd.DataFrame.from_dict(dicionario_convi, orient='index')


df_convi

,Item 1,Item 2,Convicção
0,'baking needs','bread and cake',1.297136
1,milk-cream,'bread and cake',1.368496
2,fruit,'bread and cake',1.303425
3,vegetables,'bread and cake',1.251888
4,vegetables,fruit,1.413129


A convicção é uma métrica que indica a dependência entre os itens, dentre as convicções fortes da análise temos: creme de leite, pão e bolos, sendo equivalente a 1.36; frutas, pão e bolos, sendo 1.3 e vegetais e frutas sendo a mais forte, 1.4. Isto é, esses itens aparecem frequentemente juntos e com baixas chances de serem comprados separados.

# Regra de associação (A,B)=>C

---
## Calculando a uniao de ABC

In [ ]:
def calcula_uni_ABC(A, B, C):
    uniao_ABC = smt.value_counts([A, B, C]).get((1, 1, 1), 0)
    return uniao_ABC / total
dicionario_uniao_ABC = {}
for i, item_i in enumerate(dicionario_suportes.keys()):
        for j, item_j in enumerate(dicionario_suportes.keys()):
          for k, item_k in enumerate(dicionario_suportes.keys()):
            if i > j and j > k:  # Garante que não estamos repetindo combinações
                try:
                    uniao_ABC = calcula_uni_ABC(item_i, item_j, item_k)
                    if uniao_ABC != 0:
                        dicionario_uniao_ABC[len(dicionario_uniao_ABC)] = {'Item 1': item_i, 'Item 2': item_j, 'Item3': item_k, 'União': uniao_ABC}
                except ValueError:
                    pass
df_uniao_ABC=pd.DataFrame.from_dict(dicionario_uniao_ABC, orient='index')
df_uniao_ABC

,Item 1,Item 2,Item3,União
0,milk-cream,'baking needs','bread and cake',0.341474
1,fruit,'baking needs','bread and cake',0.338016
2,fruit,milk-cream,'bread and cake',0.363951
3,fruit,milk-cream,'baking needs',0.295008
4,vegetables,'baking needs','bread and cake',0.342771
5,vegetables,milk-cream,'bread and cake',0.358332
6,vegetables,milk-cream,'baking needs',0.300843
7,vegetables,fruit,'bread and cake',0.387076
8,vegetables,fruit,'baking needs',0.321807
9,vegetables,fruit,milk-cream,0.339529


---
## Confiança {A,B}=> {C}




In [ ]:
def calcula_conf_ABC (A, B, C):
  return calcula_uni_ABC(A, B, C) / calcula_uni(A,B)


dicionario_conf_ABC={}
for i, item_i in enumerate(dicionario_suportes.keys()):
        for j, item_j in enumerate(dicionario_suportes.keys()):
          for k, item_k in enumerate(dicionario_suportes.keys()):
            if i > j and j > k:  # Garante que não estamos repetindo combinações
                try:
                    conf_ABC= calcula_conf_ABC(item_i, item_j, item_k)
                    if conf_ABC > CONF_min:
                        dicionario_conf_ABC[len(dicionario_conf_ABC)] = {'Item 1': item_i, 'Item 2': item_j, 'Item3': item_k, 'Confiança': conf_ABC}
                except ValueError:
                    pass
df_conf_ABC=pd.DataFrame.from_dict(dicionario_conf_ABC, orient='index')
df_conf_ABC

,Item 1,Item 2,Item3,Confiança
0,milk-cream,'baking needs','bread and cake',0.828526
1,fruit,'baking needs','bread and cake',0.823158
2,fruit,milk-cream,'bread and cake',0.826300
3,fruit,milk-cream,'baking needs',0.669774
4,vegetables,'baking needs','bread and cake',0.813751
5,vegetables,milk-cream,'bread and cake',0.818765
6,vegetables,milk-cream,'baking needs',0.687407
7,vegetables,fruit,'bread and cake',0.811509
8,vegetables,fruit,'baking needs',0.674671
9,vegetables,fruit,milk-cream,0.711826


Analisando os valores da Confiança{A,B}=>{C} nota-se que o valor médio da confiança aumenta, como por exemplo, em 82,85% dos casos o cliente que comprou creme de leite e itens de panificação também levou pão e bolo. O mesmo efeito acontece para frutas, itens de panificação, pão e bolo, o cliente que leva os dois primeiros itens em 82,31% dos casos acaba levando também o terceiro(pão e bolo). Quanto às frutas, creme de leite, pão e bolo esse número se mantém nesta faixa, sendo a confiança desses itens de 82,63%.
 Por outro lado, quem compra vegetais e frutas, não acaba levando itens de panificação com muita frequência, apenas 67,46% das vezes. O mesmo acontece com os clientes que levam frutas e creme de leite, os quais compram itens de panificação somente em 67% das vezes.  Dessa forma promover uma promoção entre esses itens, ou então diminuir o preço dos itens de panificação pode aumentar as vendas.


---
## Lift {A,B} => {C}

In [ ]:
def calcula_lift_ABC(A, B, C):
   return calcula_uni_ABC(A,B,C) / (calcula_suporte(A)*calcula_suporte(B)*calcula_suporte(C))



dicionario_lift_ABC = {}
for i, item_i in enumerate(dicionario_suportes.keys()):
        for j, item_j in enumerate(dicionario_suportes.keys()):
            for k, item_k in enumerate(dicionario_suportes.keys()):
              if i > j and j > k:  # Garante que não estamos repetindo combinações
                try:
                    lift_ABC = calcula_lift_ABC(item_i, item_j, item_k)
                    if lift_ABC >= LIFT_min:
                        dicionario_lift_ABC[len(dicionario_lift_ABC)] = {'Item 1': item_i, 'Item 2': item_j, 'Item3': item_k, 'Lift': lift_ABC}
                except ValueError:
                    pass
df_lift_ABC=pd.DataFrame.from_dict(dicionario_lift_ABC, orient='index')
df_lift_ABC

,Item 1,Item 2,Item3,Lift
0,milk-cream,'baking needs','bread and cake',1.236603
1,fruit,'baking needs','bread and cake',1.214576
2,fruit,milk-cream,'bread and cake',1.243690
3,fruit,milk-cream,'baking needs',1.201061
4,vegetables,'baking needs','bread and cake',1.232077
5,vegetables,milk-cream,'bread and cake',1.224902
6,vegetables,milk-cream,'baking needs',1.225232
7,vegetables,fruit,'bread and cake',1.312886
8,vegetables,fruit,'baking needs',1.300434
9,vegetables,fruit,milk-cream,1.304825


Nesta seção, podemos analisar as relações de associação, como Lift{A,B}=>{C}, e algumas regras bastante sólidas. Por exemplo, quando observamos a compra de vegetais, frutas, pão e bolo, notamos um lift de 1.31, o que sugere que os clientes que compram vegetais e frutas têm 31% mais probabilidade de também comprar pão e bolo. Conjuntos como {vegetais, frutas, itens de panificação} e {vegetais, frutas e creme de leite} exibem lifts próximos a 1.3.

---
## Convicção {A,B}=>{C}

In [ ]:
def calcula_conv_ABC(A,B,C):
  return (1-calcula_suporte(C)) / (1-calcula_conf_ABC(A,B,C))
dicionario_conv_ABC = {}
for i, item_i in enumerate(dicionario_suportes.keys()):
        for j, item_j in enumerate(dicionario_suportes.keys()):
            for k, item_k in enumerate(dicionario_suportes.keys()):
              if i > j and j > k:  # Garante que não estamos repetindo combinações
                try:
                    conv_ABC = calcula_conv_ABC(item_i, item_j, item_k)
                    if conv_ABC >= CONVI_min:
                        dicionario_conv_ABC[len(dicionario_conv_ABC)] = {'Item 1': item_i, 'Item 2': item_j, 'Item3': item_k, 'Convicção': conv_ABC}
                except ValueError:
                    pass
df_conv_ABC=pd.DataFrame.from_dict(dicionario_conv_ABC, orient='index')
df_conv_ABC

,Item 1,Item 2,Item3,Convicção
0,milk-cream,'baking needs','bread and cake',1.634720
1,fruit,'baking needs','bread and cake',1.585093
2,fruit,milk-cream,'bread and cake',1.613769
3,vegetables,'baking needs','bread and cake',1.505032
4,vegetables,milk-cream,'bread and cake',1.546676
5,vegetables,milk-cream,'baking needs',1.266623
6,vegetables,fruit,'bread and cake',1.487132
7,vegetables,fruit,'baking needs',1.217037
8,vegetables,fruit,milk-cream,1.265955


Na análise das convicções {A,B} => {C}, destacam-se valores mais elevados, indicando uma forte dependência entre os itens. Por exemplo, o conjunto {creme de leite, itens de panificação, pão e bolo} exibe uma convicção de 1.63. Isso significa que a presença de creme de leite e itens de panificação aumenta a probabilidade de "pão e bolo" estar presente em 63% em comparação com o que seria esperado ao acaso, destacando a dependência desses itens. Outros conjuntos também apresentam regras fortes, como {frutas, itens de panificação, pão e bolo} com uma convicção de 1.58, {frutas, creme de leite, pão e bolo} com 1.61, e {vegetais, creme de leite, pão e bolo} com 1.54.

--------
# Regra de associação (A,B,C)=>D

## Calculando a uniao de ABCD

In [ ]:
def calcula_uni_ABCD(A, B, C, D):
    uniao_ABCD = smt.value_counts([A, B, C, D]).get((1, 1, 1, 1), 0)
    return uniao_ABCD / total
dicionario_uniao_ABCD = {}
for i, item_i in enumerate(dicionario_suportes.keys()):
        for j, item_j in enumerate(dicionario_suportes.keys()):
          for k, item_k in enumerate(dicionario_suportes.keys()):
            for g, item_g in enumerate(dicionario_suportes.keys()):
              if i > j and j > k and k > g:  # Garante que não estamos repetindo combinações
                try:
                    uniao_ABCD = calcula_uni_ABCD(item_i, item_j, item_k,item_g)
                    if uniao_ABCD != 0:
                        dicionario_uniao_ABCD[len(dicionario_uniao_ABCD)] = {'Item 1': item_i, 'Item 2': item_j, 'Item 3': item_k, "Item 4":item_g, 'União': uniao_ABCD}
                except ValueError:
                    pass
df_uniao_ABCD=pd.DataFrame.from_dict(dicionario_uniao_ABCD, orient='index')
df_uniao_ABCD

,Item 1,Item 2,Item 3,Item 4,União
0,fruit,milk-cream,'baking needs','bread and cake',0.250919
1,vegetables,milk-cream,'baking needs','bread and cake',0.252648
2,vegetables,fruit,'baking needs','bread and cake',0.271234
3,vegetables,fruit,milk-cream,'bread and cake',0.283337
4,vegetables,fruit,milk-cream,'baking needs',0.237087


---
## Confiança {A,B,C} => {D}

In [ ]:
def calcula_conf_ABCD (A, B, C, D):
  return calcula_uni_ABCD(A, B, C, D) / calcula_uni_ABC(A,B,C)


dicionario_conf_ABCD={}
for i, item_i in enumerate(dicionario_suportes.keys()):
        for j, item_j in enumerate(dicionario_suportes.keys()):
          for k, item_k in enumerate(dicionario_suportes.keys()):
            for g, item_g in enumerate(dicionario_suportes.keys()):
              if i > j and j > k and k > g:  # Garante que não estamos repetindo combinações
                try:
                    conf_ABCD = calcula_conf_ABCD(item_i, item_j, item_k,item_g)
                    if conf_ABCD >= CONF_min:
                        dicionario_conf_ABCD[len(dicionario_conf_ABCD)] = {'Item 1': item_i, 'Item 2': item_j, 'Item 3': item_k, "Item 4":item_g, 'Confiança': conf_ABCD}
                except ValueError:
                    pass
df_conf_ABCD=pd.DataFrame.from_dict(dicionario_conf_ABCD, orient='index')
df_conf_ABCD

,Item 1,Item 2,Item 3,Item 4,Confiança
0,fruit,milk-cream,'baking needs','bread and cake',0.850549
1,vegetables,milk-cream,'baking needs','bread and cake',0.839799
2,vegetables,fruit,'baking needs','bread and cake',0.842848
3,vegetables,fruit,milk-cream,'bread and cake',0.834500


Ao analisar a confiança, podemos ver que os clientes que compraram frutas(“fruits”), creme de leite(“milk-cream”), itens de panificação(“baking needs”) compraram também pão e bolo (“bread and cake”) 85% das vezes, sendo essa confiança  maior do que do conjunto {vegetais (“vegetables”); creme de leite(“milk-cream”);itens de panificação(“baking needs”); pão e bolo (“bread and cake”) - 83% }. Quando analisamos a compra de vegetais (“vegetables”), frutas(“fruits”), itens de panificação(“baking needs”) e pão e bolo (“bread and cake”) o valor corresponde a 84%, à vista disso outra relação forte é de, vegetais (“vegetables”), frutas(“fruits”), creme de leite(“milk-cream”) e pão e bolo (“bread and cake”) com 83%. Essas análises indicam que existe uma forte associação entre todos os quatro itens juntos. Os valores de confiança são altos em todas as combinações. Essa informação é útil para estratégias de disposição de produtos em uma loja e marketing, dado que indica a possibilidade de promover esses produtos como um conjunto por serem frequentemente comprados em conjunto.

-----
## Lift {A,B,C} => {D}

In [ ]:
def calcula_lift_ABCD(A, B, C, D):
   return calcula_uni_ABCD(A,B,C,D) / (calcula_suporte(A)*calcula_suporte(B)*calcula_suporte(C)*calcula_suporte(D))



dicionario_lift_ABCD = {}
for i, item_i in enumerate(dicionario_suportes.keys()):
        for j, item_j in enumerate(dicionario_suportes.keys()):
            for k, item_k in enumerate(dicionario_suportes.keys()):
              for g, item_g in enumerate(dicionario_suportes.keys()):
                if i > j and j > k and k > g:  # Garante que não estamos repetindo combinações
                  try:
                    lift_ABCD = calcula_lift_ABCD(item_i, item_j, item_k,item_g)
                    if lift_ABCD >= LIFT_min:
                        dicionario_lift_ABCD[len(dicionario_lift_ABCD)] = {'Item 1': item_i, 'Item 2': item_j, 'Item 3': item_k, "Item 4":item_g, 'Lift': lift_ABCD}
                  except ValueError:
                    pass
df_lift_ABCD=pd.DataFrame.from_dict(dicionario_lift_ABCD, orient='index')
df_lift_ABCD

,Item 1,Item 2,Item 3,Item 4,Lift
0,fruit,milk-cream,'baking needs','bread and cake',1.419450
1,vegetables,milk-cream,'baking needs','bread and cake',1.429713
2,vegetables,fruit,'baking needs','bread and cake',1.522975
3,vegetables,fruit,milk-cream,'bread and cake',1.512983
4,vegetables,fruit,milk-cream,'baking needs',1.508344


Quanto maior o valor de lift, mais forte é a associação, em resumo, todos os lifts são maiores que 1, indicando associação positiva entre os itens. Assim, podemos concluir que há uma associação moderada a forte entre os itens mencionados, com a associação mais forte ocorrendo quando vegetais, frutas, itens de panificação e pão e bolo são comprados juntos sendo 1.52, seguido pela presença de vegetais, frutas, creme de leite e pão e bolo, de 1.51

----
## Convicção {A,B,C}=>{D}

In [ ]:
def calcula_conv_ABCD(A,B,C,D):
  return (1-calcula_suporte(D)) / (1-calcula_conf_ABCD(A,B,C,D))
dicionario_conv_ABCD = {}
for i, item_i in enumerate(dicionario_suportes.keys()):
        for j, item_j in enumerate(dicionario_suportes.keys()):
            for k, item_k in enumerate(dicionario_suportes.keys()):
              for g, item_g in enumerate(dicionario_suportes.keys()):
                if i > j and j > k and k > g:  # Garante que não estamos repetindo combinações
                  try:
                    conv_ABCD = calcula_conv_ABCD(item_i, item_j, item_k,item_g)
                    if conv_ABCD >= CONVI_min:
                        dicionario_conv_ABCD[len(dicionario_conv_ABCD)] = {'Item 1': item_i, 'Item 2': item_j, 'Item 3': item_k, "Item 4":item_g, 'Convicção': conv_ABCD}
                  except ValueError:
                    pass
df_conv_ABCD=pd.DataFrame.from_dict(dicionario_conv_ABCD, orient='index')
df_conv_ABCD

,Item 1,Item 2,Item 3,Item 4,Convicção
0,fruit,milk-cream,'baking needs','bread and cake',1.875612
1,vegetables,milk-cream,'baking needs','bread and cake',1.749745
2,vegetables,fruit,'baking needs','bread and cake',1.783690
3,vegetables,fruit,milk-cream,'bread and cake',1.693727
4,vegetables,fruit,milk-cream,'baking needs',1.312272


A dependência mais forte entre os itens é apresentada na compra de frutas, creme de leite, itens de panificação e pão e bolo, de 1.87, seguida de convicções fortes como vegetais, frutas, itens de panificação e pão e bolo com o valor de 1.78, assim como  vegetais, creme de leite, itens de panificação e pão e bolo apresentam 1.74. Como resultado, dificilmente esses itens serão comprados separados por conta do alto valor da convicção.

# Regra de associação {A,B}=>{C,D}

## Calculando a união

In [ ]:
def calcula_uni_AB_CD(A, B, C, D):
    uniao_AB_CD = smt.value_counts([A, B, C, D]).get((1, 1, 1, 1), 0)
    return uniao_AB_CD / total
dicionario_uniao_AB_CD = {}
for i, item_i in enumerate(dicionario_suportes.keys()):
        for j, item_j in enumerate(dicionario_suportes.keys()):
          for k, item_k in enumerate(dicionario_suportes.keys()):
            for g, item_g in enumerate(dicionario_suportes.keys()):
              if i > j and j > k and k > g:  # Garante que não estamos repetindo combinações
                try:
                    uniao_AB_CD = calcula_uni_AB_CD(item_i, item_j, item_k,item_g)
                    if uniao_AB_CD != 0:
                        dicionario_uniao_AB_CD[len(dicionario_uniao_AB_CD)] = {'Item 1': item_i, 'Item 2': item_j, 'Item 3': item_k, "Item 4":item_g, 'União': uniao_AB_CD}
                except ValueError:
                    pass
df_uniao_AB_CD=pd.DataFrame.from_dict(dicionario_uniao_AB_CD, orient='index')
df_uniao_AB_CD

,Item 1,Item 2,Item 3,Item 4,União
0,fruit,milk-cream,'baking needs','bread and cake',0.250919
1,vegetables,milk-cream,'baking needs','bread and cake',0.252648
2,vegetables,fruit,'baking needs','bread and cake',0.271234
3,vegetables,fruit,milk-cream,'bread and cake',0.283337
4,vegetables,fruit,milk-cream,'baking needs',0.237087


---
## Confiança {A,B}=>{C,D}

In [ ]:
def calcula_conf_AB_CD (A, B, C, D):
  return calcula_uni_AB_CD(A, B, C, D) / calcula_uni(A,B)


dicionario_conf_AB_CD={}
for i, item_i in enumerate(dicionario_suportes.keys()):
        for j, item_j in enumerate(dicionario_suportes.keys()):
          for k, item_k in enumerate(dicionario_suportes.keys()):
            for g, item_g in enumerate(dicionario_suportes.keys()):
              if i > j and j > k and k > g:  # Garante que não estamos repetindo combinações
                try:
                    conf_AB_CD = calcula_conf_AB_CD(item_i, item_j, item_k,item_g)
                    if conf_AB_CD >= CONF_min:
                        dicionario_conf_AB_CD[len(dicionario_conf_AB_CD)] = {'Item 1 e Item 2': (item_i, item_j),'Item 3 e Item 4': (item_k, item_g),'Confiança': conf_AB_CD}
                except ValueError:
                    pass
df_conf_AB_CD=pd.DataFrame.from_dict(dicionario_conf_AB_CD, orient='index')
df_conf_AB_CD

,Item 1 e Item 2,Item 3 e Item 4,Confiança
0,"(fruit, milk-cream)","('baking needs', 'bread and cake')",0.569676
1,"(vegetables, milk-cream)","('baking needs', 'bread and cake')",0.577284
2,"(vegetables, fruit)","('baking needs', 'bread and cake')",0.568645
3,"(vegetables, fruit)","(milk-cream, 'bread and cake')",0.594019



Ao analisar a Confiança{A,B}=>{C,D} notamos que os valores das confianças quando comparadas com Confiança{A,B,C}=>{D} diminuem, como por nesse caso a confiança{fruit, milk-cream}=>{baking needs, bread and cake} vale 0.56, entretanto a confiança{fruit, milk-cream, baking needs}=>{bread and cake} vale 0.85. Isso acontece pela inclusão de mais itens no antecedente, adicionar mais restrições pode reduzir o número de transações que satisfazem todas as condições impostas pela regra.


---
## Lift {A,B}=>{C,D}

In [ ]:
def calcula_lift_AB_CD(A, B, C, D):
   return calcula_uni_AB_CD(A,B,C,D) / (calcula_suporte(A)*calcula_suporte(B)*calcula_suporte(C)*calcula_suporte(D))



dicionario_lift_AB_CD = {}
for i, item_i in enumerate(dicionario_suportes.keys()):
        for j, item_j in enumerate(dicionario_suportes.keys()):
            for k, item_k in enumerate(dicionario_suportes.keys()):
              for g, item_g in enumerate(dicionario_suportes.keys()):
                if i > j and j > k and k > g:  # Garante que não estamos repetindo combinações
                  try:
                    lift_AB_CD = calcula_lift_AB_CD(item_i, item_j, item_k,item_g)
                    if lift_AB_CD >= LIFT_min:
                        dicionario_lift_AB_CD[len(dicionario_lift_AB_CD)] = {'Item 1 e Item 2': (item_i, item_j),'Item 3 e Item 4': (item_k, item_g),'Lift': lift_AB_CD}
                  except ValueError:
                    pass
df_lift_AB_CD=pd.DataFrame.from_dict(dicionario_lift_AB_CD, orient='index')
df_lift_AB_CD

,Item 1 e Item 2,Item 3 e Item 4,Lift
0,"(fruit, milk-cream)","('baking needs', 'bread and cake')",1.419450
1,"(vegetables, milk-cream)","('baking needs', 'bread and cake')",1.429713
2,"(vegetables, fruit)","('baking needs', 'bread and cake')",1.522975
3,"(vegetables, fruit)","(milk-cream, 'bread and cake')",1.512983
4,"(vegetables, fruit)","(milk-cream, 'baking needs')",1.508344


Regras fortes para o Lift são: (vegetais e frutas)=>(creme de leite, pão e bolo) sendo que esses itens possuem 51% de chances de serem encontrados juntos. O mesmo vale para: (vegetais e frutas)=>(itens de panificação, pão e bolo); (vegetais e frutas)=>(creme de leite, itens de panificação).

---
## Convicção {A,B}=>{C,D}

In [ ]:
def calcula_conv_AB_CD(A,B,C,D):
  return (1-calcula_uni(C,D)) / (1-calcula_conf_ABCD(A,B,C,D))
dicionario_conv_AB_CD = {}
for i, item_i in enumerate(dicionario_suportes.keys()):
        for j, item_j in enumerate(dicionario_suportes.keys()):
            for k, item_k in enumerate(dicionario_suportes.keys()):
              for g, item_g in enumerate(dicionario_suportes.keys()):
                if i > j and j > k and k > g:  # Garante que não estamos repetindo combinações
                  try:
                    conv_AB_CD = calcula_conv_AB_CD(item_i, item_j, item_k,item_g)
                    if conv_AB_CD >= CONVI_min:
                        dicionario_conv_AB_CD[len(dicionario_conv_AB_CD)] = {'Item 1 e Item 2': (item_i, item_j),'Item 3 e Item 4': (item_k, item_g),'Convicção': conv_AB_CD}
                  except ValueError:
                    pass
df_conv_AB_CD=pd.DataFrame.from_dict(dicionario_conv_AB_CD, orient='index')
df_conv_AB_CD

,Item 1 e Item 2,Item 3 e Item 4,Convicção
0,"(fruit, milk-cream)","('baking needs', 'bread and cake')",3.522737
1,"(vegetables, milk-cream)","('baking needs', 'bread and cake')",3.286337
2,"(vegetables, fruit)","('baking needs', 'bread and cake')",3.350091
3,"(vegetables, fruit)","(milk-cream, 'bread and cake')",2.990466
4,"(vegetables, fruit)","(milk-cream, 'baking needs')",1.948351


Já para a Convicção{A,B}=>{C,D} existem regras extremamentes fortes como: (frutas, creme de leite)=>(itens de panificação, pão e bolo - 3.522); (vegetais, creme de leite)=>(itens de panificação, pão e bolo - 3.28); (vegetais, frutas)=>(itens de panificação, pão e bolo - 3.35).

---
# Conclusões finais

Durante toda a análise, foi constatado que as regras mais consistentes envolvem frutas, itens de panificação, vegetais, pão e bolo. Isso sugere que o supermercado pode se beneficiar ao tomar decisões estratégicas em relação a esses itens. Por exemplo, criar promoções específicas que incentivem a compra conjunta desses produtos ou até mesmo posicionar esses itens estrategicamente em locais que incentivem os clientes a percorrerem todo o supermercado, aumentando assim as chances de compra de outros produtos. Essas são apenas algumas das possíveis abordagens que o supermercado pode considerar com base nas associações identificadas.